# Static Data Prep
Prepares dataframe of static features (Demographics, diagnoses, admission info, DRG info, height, and weight).

In [388]:
import pickle
import numpy as np
import pandas as pd

# Load data

In [389]:
# Directly accessed
admissions = pd.read_csv('mimic_iii_data/ADMISSIONS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
patients   = pd.read_csv('mimic_iii_data/PATIENTS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
dx         = pd.read_csv('mimic_iii_data/DIAGNOSES_ICD.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
drg        = pd.read_csv('mimic_iii_data/DRGCODES.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
lab        = pd.read_csv('mimic_iii_data/LABEVENTS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
rx         = pd.read_csv('mimic_iii_data/PRESCRIPTIONS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
rx.DRUG    = rx.DRUG.map(lambda x: x.lower().replace('*',' '))

# Obtained via SQL queries
ht_wt = pd.read_csv('mimic_iii_data/ht_wt.csv')
ht_neo= pd.read_csv('mimic_iii_data/len.csv')

drg.drop_duplicates(inplace=True)
drg.DESCRIPTION = drg.DESCRIPTION.map(lambda x: x.lower() if pd.notna(x) else x)

/Users/Bova/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [390]:
%run functions.ipynb
with open(f'maps.pkl', 'rb') as f:
    maps = pickle.load(f)

# Create DF

In [392]:
# Create DF
X = admissions[['HADM_ID','ADMISSION_TYPE','ADMISSION_LOCATION','LANGUAGE','ETHNICITY',
                'DIAGNOSIS','HOSPITAL_EXPIRE_FLAG','ADMITTIME','DISCHTIME','INSURANCE']].copy()

### Add Columns

In [393]:
# Length of Stay
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME)
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME)
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME)

X['LOS'] = (admissions.DISCHTIME - admissions.ADMITTIME).dt.days
# remove DOA patients
X = X[X.LOS>0]

In [394]:
# Patient age and gender
T = pd.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME']],
             patients[['SUBJECT_ID','DOB','GENDER']],
             on='SUBJECT_ID')

T_old    = T[T.DOB < '2000-01-01'] # all patients over 89 have age>300
T        = T[T.DOB > '2000-01-01']

T['AGE'] =((pd.to_datetime(T.ADMITTIME).dt.date - 
            pd.to_datetime(T.DOB).dt.date).values//(3.1689*10**16)).astype(int)
T_old['AGE'] = 90
T = T.append(T_old)
X = X.merge(T[['HADM_ID','AGE','GENDER']],on='HADM_ID')
X.GENDER = X.GENDER.map(lambda x: 0 if x=='M' else 1)

In [395]:
# ICD9 codes (diagnosis 1 used to impute missing DIAGNOSIS values)
X = X.merge(dx[['HADM_ID','ICD9_CODE']][dx.SEQ_NUM==1],on='HADM_ID')\
     .rename(columns = {'ICD9_CODE':f'DIAGNOSIS_1'})
    
X = X.merge(dx.groupby('HADM_ID').SEQ_NUM.max(),on='HADM_ID')\
     .rename(columns = {'SEQ_NUM':f'N_DIAGNOSES'})           \
     .set_index('HADM_ID')

# Fill missing values

In [396]:
for col in X.columns:
    print(f'{col}:\n\t{X[col].isna().sum()}')

ADMISSION_TYPE:
	0
ADMISSION_LOCATION:
	0
LANGUAGE:
	24369
ETHNICITY:
	0
DIAGNOSIS:
	3
HOSPITAL_EXPIRE_FLAG:
	0
ADMITTIME:
	0
DISCHTIME:
	0
INSURANCE:
	0
LOS:
	0
AGE:
	0
GENDER:
	0
DIAGNOSIS_1:
	0
N_DIAGNOSES:
	0


### Fill Missing dx

In [397]:
# Impute using missing diagnosis map
X.DIAGNOSIS = X[['DIAGNOSIS','DIAGNOSIS_1']].set_index('DIAGNOSIS_1').DIAGNOSIS\
                                            .fillna(maps['diagnosis_na']).values

### Fill Missing Language

In [398]:
# missing values assumed to be English. Map to binary, indicating presence of language barrier
X.LANGUAGE = X.LANGUAGE.map(lambda x: 1 if x=='ENGL' else 0) 

In [399]:
# check for any remaining NANs
X.isna().sum().sum()

0

## Number Previous Admissions

In [400]:
admissions = admissions.sort_values(by='ADMITTIME')
admissions['N_ADMISSIONS'] = admissions.groupby(['SUBJECT_ID']).cumcount()
X = X.join(admissions[['HADM_ID','N_ADMISSIONS']].set_index('HADM_ID'))

# Narrow categorical
### Broaden ethnicity categories

In [401]:
# narrow ethnicity categories (english vs. non-english speaking)
X.ETHNICITY = X.ETHNICITY.map(maps['ethnicity']) #drop unknown after dummies created

### Broaden Admission Location categories

In [402]:
X.ADMISSION_LOCATION = X.ADMISSION_LOCATION.map(maps['admission_location'])

### Map ICD9 codes

In [403]:
X = add_columns(X,dx,maps['icd'],'ICD9_CODE').drop(['SUBJECT_ID','SEQ_NUM'],axis=1)

GI_INFECTION
TB
BACTERIAL_INFECTION_OTHER
SEPSIS
HIV
VIRAL_CNS
VIRAL_HEPATITIS
VIRUS_OTHER
CHLAMIDIA
ARTHROPOD_BORN_INFECTION
SYPHILIS
DERMATOPHYTOSIS
CANDIDIASIS
MYCOSES_OTHER
HELMINTHIASIS
INFECTION_OTHER
SARCOIDOSIS
POLIO
ORAL_PHARYNGERAL_CANCER
ESOPHAGUS_STOMACH_CANCER
INTESTINAL_CANCER
LIVER_CANCER
GALLBLADDER_CANCER
PANCREATIC_CANCER
GI_CANCER_OTHER
UPPPER_RESPIRATORY_CANCER
LUNG_CANCER
RESPIRATORY_CANCER_OTHER
BONE_CONNECTIVE_TISSUE_CANCER
SKIN_CANCER
BREAST_CANCER
KAPOSIS_SARCOMA
UTERINE_CANCER
OVARIAN_CANCER
PROSTATE_CANCER
MALE_REPRO_CANCER_OTHER
BLADDER_CANCER
KIDNEY_CANCER
NEURO_CANCER_OTHER
BRAIN_CANCER
THYROID_CANCER
ENDOCRINE_CANCER_OTHER
CANCER_OTHER
METS_TO_LYMPH_NODES
METS_TO_RESP_GI
METS_TO_OTHER
LYMPHATIC_CANCER
HOGKINS_LYMPHOMA
HEME_CANCER_OTHER
MULTIPLE_MYELOMA
LEUKEMIA
NEUROENDOCRINE_TUMORS
BENIGN_GI
BENIGN_RESPIRATORY
BENIGN_OTHER
BENIGN_BRAIN
BENIGN_ENDOCRINE
UTERINE_LEIOMYOMA
HEMANGIOMA_LYMPHANGIOMA
CARCINOMA_IN_SITU
GOITER
THYROTOXICOSIS
HYPOTHYROIDISM
THYROI

### Pull Keywords from DRGs

In [404]:
# Pull tags from DRGs using DRG map
tags = ['COMA','SEVERE','VENTILATED','SX','TRANSPLANT','FAILURE','CANCER','ID','NEONATE']

drg_copy = drg.copy()
drg_copy.DESCRIPTION = drg.DESCRIPTION.map(maps['drg'])
for tag in tags:
    print(tag)
    drg_copy['TAG_'+tag] = drg_copy.DESCRIPTION.map(lambda x: get_descriptor(x,tag), na_action='ignore')
    drg_copy.DESCRIPTION = drg_copy.DESCRIPTION.map(lambda x: strip_descriptor(x,tag), na_action='ignore')

tags = ['TAG_'+tag for tag in tags]
drg_copy  = drg_copy[tags+['DESCRIPTION','HADM_ID']].drop_duplicates()
drg_copy  = pd.get_dummies(drg_copy,prefix='TAG',columns=['DESCRIPTION'])\
              .groupby('HADM_ID').sum().applymap(lambda x: 1 if x>=1 else 0)
drg_copy.drop('TAG_OTHER',axis=1,inplace=True)
X = X.merge(drg_copy,on='HADM_ID',how='left')
X[drg_copy.columns] = X[drg_copy.columns].fillna(0)

COMA
SEVERE
VENTILATED
SX
TRANSPLANT
FAILURE
CANCER
ID
NEONATE


In [405]:
# Some tags from diagnosis and DRGs overlap - assumed 1 if either is 1
overlap = ['STROKE','TRANSPLANT','NEONATE','DM','ELECTROLYTE_DX','HTN','MI','OD','PSYCH','TRAUMA','SEPSIS']

for d,regx in maps['diagnoses'].items():
    if d in overlap:
        X[f'{d}'] = X.DIAGNOSIS.str.match(regx).fillna(0).astype(int,errors='ignore')
    else:
        X[f'TAG_{d}'] = X.DIAGNOSIS.str.match(regx).fillna(0).astype(int,errors='ignore')
        
# Merge overlapping columns
for d in overlap:
    X[f'TAG_{d}'] = X[[d,'TAG_'+d]].apply(lambda x: 1 if sum(x)>0 else 0,axis=1)
    X.drop(d,axis=1,inplace=True)

## Height

In [406]:
ht_wt = ht_wt[ht_wt.VALUENUM>0]
ht_wt.ITEMID = ht_wt.ITEMID.map(maps['ht_wt_map'])

ht = ht_wt[((ht_wt.ITEMID=='Height')       |
            (ht_wt.ITEMID=='HEIGHT_IN')    |
            (ht_wt.ITEMID=='Height (cm)')  |
            (ht_wt.ITEMID=='Height Inches')|
            (ht_wt.ITEMID=='HEIGHT_CM'))   &
           (ht_wt.VALUENUM<250)].copy()
ht['HEIGHT'] = ht.VALUENUM.apply(lambda h: fix_height(h)) # convert to cm
ht = ht[ht.HEIGHT.between(100,240)]
ht = get_first(ht) 
ht = ht[~ht.index.duplicated()] 

ht_neo['HEIGHT'] = ht_neo.apply(lambda l: l.VALUENUM*2.54 if l.VALUENUM < 25 
                                                        else l.VALUENUM, axis=1)
ht_neo = ht_neo[ht_neo.HEIGHT>25]
ht_neo = get_first(ht_neo)

ht = ht.append(ht_neo)
ht = ht[~ht.index.duplicated()]
X = X.join(ht.HEIGHT)

## Weight

In [407]:
wt_neo = ht_wt[(ht_wt.ITEMID=='Birthweight (kg)')    |
               (ht_wt.ITEMID=='Birth Weight    (kg)')|
               (ht_wt.ITEMID=='Previous Weight (kg)')|
               (ht_wt.ITEMID=='Present Weight  (oz)')|  
               (ht_wt.ITEMID=='Present Weight  (lb)')|
               (ht_wt.ITEMID=='Present Weight  (kg)')].copy()
wt_neo['WEIGHT'] = wt_neo.apply(lambda w: fix_weight(w),axis=1)
wt_neo = wt_neo[wt_neo.WEIGHT.between(0.3,20)]
wt_neo = get_first(wt_neo)

wt = ht_wt[((ht_wt.ITEMID=='Daily Weight')           |    # kg
            (ht_wt.ITEMID=='Admission Weight (Kg)')  |
            (ht_wt.ITEMID=='Admission Weight (lbs.)')|
            (ht_wt.ITEMID=='Weight Kg')              |
            (ht_wt.ITEMID=='Previous Weight')        |    # kg
            (ht_wt.ITEMID=='Previous WeightF')       |    # kg
            (ht_wt.ITEMID=='Previous Weight'))       &    # kg
            ((ht_wt.VALUENUM/2.2)<636)].copy()            # record human wt
wt = wt[wt.VALUENUM.between(0.3,1400)]
wt['WEIGHT'] = wt.apply(lambda w: w.VALUENUM/2.2 if w.ITEMID=='Admission Weight (lbs.)' 
                                               else w.VALUENUM, axis=1)\
                 .apply(lambda w: w/2.2 if w > 400 else w)
wt = get_first(wt)
wt = wt.append(wt_neo)
wt = wt[~wt.index.duplicated()]

X  = X.join(wt.WEIGHT)

## Drop any 1-level and superfluous columns

In [408]:
one_level_columns = [c for c in X.columns if len(X[c].unique())<2]
print(one_level_columns)

drop = ['DIAGNOSIS'  ,
        'DIAGNOSIS_1']

X = X.drop(one_level_columns+drop,axis=1)

['LEUKEMIA', 'BENIGN_GI', 'PREGNANCY_COMPLICATION_OTHER']


## Impute missing weight and height

In [409]:
# Temporarilly remove columns not used in imputing
no_imputation = ['HOSPITAL_EXPIRE_FLAG','ADMITTIME','DISCHTIME','LOS']
temp = X[no_imputation]
X = X.drop(no_imputation,axis=1)
X = pd.get_dummies(X,columns=['ADMISSION_TYPE','ADMISSION_LOCATION','ETHNICITY','INSURANCE'])

In [410]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# convert to meters so height and weight can have same min value (0.25 kg and 0.25 m)
X.HEIGHT = X.HEIGHT/100
try:
    imputer = pickle.load( open('ht_wt_imputer.pkl', 'rb' ))
except OSError:
    imputer = IterativeImputer(min_value    = 0.25   ,
                              random_state = 1984,
                              verbose      = 2   )
    imputer.fit(X)
    pickle.dump(imputer,open('ht_wt_imputer.pkl', 'wb'))
    
X = pd.DataFrame(imputer.transform(X),
                 columns = X.columns ,
                 index   = X.index   )
X.HEIGHT = X.HEIGHT*100 # Prefer cm

[IterativeImputer] Completing matrix with shape (56676, 438)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1299.82
[IterativeImputer] Change: 91.1581965810972, scaled tolerance: 0.5686363636363636 
[IterativeImputer] Ending imputation round 2/10, elapsed time 2584.40
[IterativeImputer] Change: 7.133564434151822, scaled tolerance: 0.5686363636363636 
[IterativeImputer] Ending imputation round 3/10, elapsed time 4017.65
[IterativeImputer] Change: 1.1610904495760201, scaled tolerance: 0.5686363636363636 
[IterativeImputer] Ending imputation round 4/10, elapsed time 5377.37
[IterativeImputer] Change: 0.4132630128793908, scaled tolerance: 0.5686363636363636 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (56676, 438)
[IterativeImputer] Ending imputation round 1/4, elapsed time 0.47
[IterativeImputer] Ending imputation round 2/4, elapsed time 0.92
[IterativeImputer] Ending imputation round 3/4, elapsed time 1.33
[Iterative

## Add BMI

In [411]:
X = X.join(temp)
X['BMI'] = X.WEIGHT/(X.HEIGHT/100)**2

# Save

In [412]:
X.to_csv('X_y_static.csv')